# GENERADOR DE ESCRITOS SAGRADOS DE LA SAGRADA BIBLIA

## 1. Instalar librerías cuda

In [1]:
#!pip list
!nvcc --version  #Version de CUDA en la maquina virtual

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


## 2. Importacion de librerías

In [2]:
import tensorflow as tf
import timeit               #para medir tiempos
import numpy as np
import pandas as pd 
import os
import time
import sys

## 3. Uso de GPU

In [25]:
print("Tensorflow Version: ", tf.__version__)
print("Dispositivos disponibles para entrenar: ", tf.config.list_physical_devices())
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Encontrada la GPU: {}'.format(device_name))

Tensorflow Version:  2.7.0
Dispositivos disponibles para entrenar:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Encontrada la GPU: /device:GPU:0


## 4. GPU VS CPU

In [5]:
def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

In [6]:
cpu()  #ejecutamos entrenamiento con CPU
gpu()  #ejecutamos entrenamiento con GPU
# Run the op several times.
print('TIEMPO (seg) para entrenar una red convolucional de 32x7x7x3 filtros sobre un randomico de 100x100x100x3 imagenes '
      '(batch x height x width x channel). suma de 10 epochs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

TIEMPO (seg) para entrenar una red convolucional de 32x7x7x3 filtros sobre un randomico de 100x100x100x3 imagenes (batch x height x width x channel). suma de 10 epochs.
CPU (s):
3.785504455000023
GPU (s):
0.05223093699999026
GPU speedup over CPU: 72x


## 5. Dejando activa la GPU

In [26]:
#tf.device('/gpu:0') #activando la CPU
tf.device('/device:GPU:0') #activando la GPU

## 6. Descarga y preproceso de datos

In [8]:
#https://raw.githubusercontent.com/JuanDiegoCastellanos/deep_learning/main/Datasets/Biblia_Reina-Valera_1909.txt
fileDL= tf.keras.utils.get_file('Biblia_Reina-Valera_1909.txt','https://raw.githubusercontent.com/JuanDiegoCastellanos/deep_learning/main/Datasets/Biblia_Reina-Valera_1909.txt')
texto = open(fileDL, 'rb').read().decode(encoding='utf-8')


3112960/3096688 [==============================] - 0s 0us/step


### 6.1 Normalización del texto
1. pasar todo a minusculas
2. convertir tildes a vocales sin tilde
3. eliminar caracteres especiales (*#!()%=)
4. eliminar saltos de linea y tab (identación)

In [ ]:
import re
from unicodedata import normalize
#pasa todo a minuscula
texto     = texto.lower()
#reemplazar tildes por letras similares sin tildes
transfor  = dict.fromkeys(map(ord, u'\u0301\u0308'), None)
texto     = normalize('NFKC', normalize('NFKD', texto).translate(transfor))
#quitar saltos de linea
texto      = texto.strip()
texto      = re.sub('\r|\n', ' ',texto)
#quitar espacios dobles
texto      = re.sub(' +', ' ', texto)
#quitando caracteres especiales
texto = re.sub(r"[^a-zA-Z0-9]+"," ",texto)
print(texto)

## 7. Comprendiendo el texto

In [10]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto)))
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de:2876717 caracteres
el texto esta compuesto de estos :36 caracteres
[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z']


## 8. Convertir Texto en Número
Las redes neuronales solo procesan valores numéricos, no letras, por tanto tenemos que traducir los caracteres a representación numérica.  Para ello crearemos dos “tablas de traducción”: una de caracteres a  números y otra de números a caracteres

In [11]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)
#-----------revisando las conversiones

#for char,_ in zip(char2idx, range(len(vocab))):
    #print(' {:4s}: {:3d},'.format(repr(char),char2idx[char]))

#pasamos todo el texto a números
texto_como_entero= np.array([char2idx[c] for c in texto])
print('texto: {}'.format(repr(texto[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: 'biblia reina valera 1909 1 en el principio crio dios los cielos y la tierra 2 y la tierra estaba des'
array([12, 19, 12, 22, 19, 11,  0, 28, 15, 19, 24, 11,  0, 32, 11, 22, 15,
       28, 11,  0,  2, 10,  1, 10,  0,  2,  0, 15, 24,  0, 15, 22,  0, 26,
       28, 19, 24, 13, 19, 26, 19, 25,  0, 13, 28, 19, 25,  0, 14, 19, 25,
       29,  0, 22, 25, 29,  0, 13, 19, 15, 22, 25, 29,  0, 34,  0, 22, 11,
        0, 30, 19, 15, 28, 28, 11,  0,  3,  0, 34,  0, 22, 11,  0, 30, 19,
       15, 28, 28, 11,  0, 15, 29, 30, 11, 12, 11,  0, 14, 15, 29])


## 9. Exportar vocablos y matriz númerica

In [12]:
rows=[]
columns=['num','vocab']
for i, voc in enumerate(vocab):
  #print(i,'-->', voc)
  rows.append([i,voc])
df= pd.DataFrame(columns=['num','vocab'],data=rows)
df.head(10)
df.to_csv('data_vocab.csv',index=False)

## 10. Preparar Datos para la RNN 

In [13]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
#cantidad de secuencia de caracteres
secu_length=50
#creamos secuencias de maximo 100 caractereres
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'biblia reina valera 1909 1 en el principio crio dio'
's los cielos y la tierra 2 y la tierra estaba desor'
'denada y vacia y las tinieblas estaban sobre la haz'
' del abismo y el espiritu de dios se movia sobre la'
' haz de las aguas 3 y dijo dios sea la luz y fue la'
' luz 4 y vio dios que la luz era buena y aparto dio'
's la luz de las tinieblas 5 y llamo dios a la luz d'
'ia y a las tinieblas llamo noche y fue la tarde y l'
'a ma ana un dia 6 y dijo dios haya expansion en med'
'io de las aguas y separe las aguas de las aguas 7 e'


## 11. Separar datos en agrupamientos (BATCHES)

In [14]:
#funcion para obtener el conjunto de datos de trainning
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)
#el dataset contiene un conjunto de parejas de secuencia de texto
#(con la representación numérica de los caracteres), donde el 
#primer componente de la pareja contiene un paquete con una secuencia 
#de 100 caracteres del texto original y la segunda su correspondiente salida, 
#también de 100 caracteres. )
for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  'biblia reina valera 1909 1 en el principio crio di'
Target data:  'iblia reina valera 1909 1 en el principio crio dio'


## 12. Mostrar el Tensor del DataSet

In [15]:
#imprimimos el tensor del dataset
print(dataset)
#Hyper-Parametros para entrenamiento  de una rede neuronal 
#   -los datos se agrupan en batch
BATCH_SIZE= 64
#    -Tamaño de memoria disponible 
BUFFER_SIZE=10000
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)
#En el tensor dataset disponemos los datos de entrenamiento
#con agrupamienttos (batches) compuestos de 64 parejas de secuencias 
#de 100 integers de 64 bits que representan el carácter correspondiente 
#en el vocabulario.

<MapDataset shapes: ((50,), (50,)), types: (tf.int64, tf.int64)>
<BatchDataset shapes: ((64, 50), (64, 50)), types: (tf.int64, tf.int64)>


## 13. Construcción de Modelo RNN
Para construir el modelo usaremos tf.keras.Sequential. Usaremos una versión mínima de RNN, que contenga solo una capa LSTM y 3 capas.



In [69]:
#como es un problema de clasificación estándar 
#para el que debemos definir la función de Lossy el optimizador.
def lossy(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def create_model(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  #En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
  #con los argumentos por defecto del optimizador Adam. 
  model.compile(optimizer='adam',
              loss='',
              metrics=['accuracy'])
  return model

def create_model_two(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  #En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
  #con los argumentos por defecto del optimizador Adam. 
  model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
  return model
vocab_size= len(vocab)
#dimensiones de los vectores que tendrá la capa.
embedding_dim= 256
#cantidad de neuronas
rnn_units=1024
#creamos nuestra red neuronal RNN
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)
#summary()para visualizar la estructura del modelo
model.summary()
#resultados=  -La capa LSTM consta más de 5 millones de parametros)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (64, None, 256)           9216      
                                                                 
 lstm_5 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_5 (Dense)             (64, None, 36)            36900     
                                                                 
Total params: 5,293,092
Trainable params: 5,293,092
Non-trainable params: 0
_________________________________________________________________


### 13.1 Creación de checkpoints
una técnica de tolerancia de fallos para procesos cuyo tiempo de ejecución es muy largo. La idea es guardar una instantánea del estado del sistema periódicamente para recuperar desde ese punto la ejecución en caso de fallo del sistema.

los crearemos en google drive para mejorar la capacidad de reentrenamiento de la red

In [17]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [28]:
checkpoint_dir='/content/gdrive/MyDrive/Colab Notebooks/RedesNeuronalesRecurrentes/checkpointsV2'
checkpoint_prefix= os.path.join(checkpoint_dir,"cp_{epoch:04d}.ckpt")


cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                               monitor='loss',
                                               verbose=1,
                                               save_weights_only=True,
                                               save_best_only=True,
                                               mode='auto')

## 14. Entrenamiento

### 14.1 Entrenando para usar checkpoints

In [29]:
EPOCHS=20
history=model.fit(dataset, 
                  epochs=EPOCHS, 
                  verbose=1,
                  callbacks=[cp_callback])

Epoch 1/20
881/881 [==============================] - ETA: 0s - loss: 1.6355 - accuracy: 0.4987
Epoch 00001: loss improved from inf to 1.63553, saving model to /content/gdrive/MyDrive/Colab Notebooks/RedesNeuronalesRecurrentes/checkpointsV2/cp_0001.ckpt
881/881 [==============================] - 83s 91ms/step - loss: 1.6355 - accuracy: 0.4987
Epoch 2/20
881/881 [==============================] - ETA: 0s - loss: 1.2320 - accuracy: 0.6143
Epoch 00002: loss improved from 1.63553 to 1.23205, saving model to /content/gdrive/MyDrive/Colab Notebooks/RedesNeuronalesRecurrentes/checkpointsV2/cp_0002.ckpt
881/881 [==============================] - 82s 92ms/step - loss: 1.2320 - accuracy: 0.6143
Epoch 3/20
881/881 [==============================] - ETA: 0s - loss: 1.1603 - accuracy: 0.6351
Epoch 00003: loss improved from 1.23205 to 1.16031, saving model to /content/gdrive/MyDrive/Colab Notebooks/RedesNeuronalesRecurrentes/checkpointsV2/cp_0003.ckpt
881/881 [==============================] - 82s 9

### 14.2 Entrenando desde un checkpoint


Desde la carpeta que optamos guardar los checkpoints

- El archivo .data es el archivo que contiene nuestras variables de entrenamiento y vamos a ir tras él.
- El archivo checkpoint, simplemente mantiene un registro de los últimos archivos de punto de control guardados


In [30]:
#creamos un modelo con iguales caracteristicas al 1° modelo
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)

#buscamos el ultimo checkpoint de entrenamiento
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

/content/gdrive/MyDrive/Colab Notebooks/RedesNeuronalesRecurrentes/checkpointsV2/cp_0020.ckpt


### 14.3 Cargando los pesos y entrenando

In [32]:
# cargamos los pesos al nuevo modelo (estos valores tienes una variación de un 10%)
model.load_weights(latest)
# continuamos el entrenamiento desde el checkpoint en que quedamos
history2=model.fit(dataset, 
                    epochs=20, 
                    verbose=1,
                    callbacks=[cp_callback])

Epoch 1/20
881/881 [==============================] - ETA: 0s - loss: 0.8937 - accuracy: 0.7213
Epoch 00001: loss improved from 0.89661 to 0.89371, saving model to /content/gdrive/MyDrive/Colab Notebooks/RedesNeuronalesRecurrentes/checkpointsV2/cp_0001.ckpt
881/881 [==============================] - 81s 91ms/step - loss: 0.8937 - accuracy: 0.7213
Epoch 2/20
881/881 [==============================] - ETA: 0s - loss: 0.8907 - accuracy: 0.7224
Epoch 00002: loss improved from 0.89371 to 0.89073, saving model to /content/gdrive/MyDrive/Colab Notebooks/RedesNeuronalesRecurrentes/checkpointsV2/cp_0002.ckpt
881/881 [==============================] - 81s 91ms/step - loss: 0.8907 - accuracy: 0.7224
Epoch 3/20
881/881 [==============================] - ETA: 0s - loss: 0.8882 - accuracy: 0.7230
Epoch 00003: loss improved from 0.89073 to 0.88825, saving model to /content/gdrive/MyDrive/Colab Notebooks/RedesNeuronalesRecurrentes/checkpointsV2/cp_0003.ckpt
881/881 [==============================] - 8

## 15. Generando texto nuevo usando la RNN

In [70]:
#creamos un modelo tomando como base el ultimo checkpoint
model = create_model_two(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [71]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores/cantidad de texto generaremos
  num_generate=500
  #convertimos el texto en números
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.2  #(0.0 a  1) entre más alta la temperatura más creatividad al modelo, pero tambien más errores ortograficos.
  model.reset_states() #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))

## 15.1 Generando Escritos sagrados

In [72]:
print(generate_text(model, start_string=u"los diez mandamientos"))

los diez mandamientos de jehova y de la tierra de egipto y las casas de sus padres y la poseeran segun sus estatutos y sus vestimentas y sus testimonios y sus mujeres a sus siervos y sus principes y sus hijas y las palabras de jehova son los que se apartan para siempre 1 y fue a mi palabra de jehova diciendo 2 hijo del hombre no quiere de aquel que se ha de parto 16 y envio jehova a moises y a aaron y a sus hijos y a toda la tierra de egipto y la tierra se esconde en la tierra de egipto 10 y la higuera que haga conf


## 16. Exportando Modelo
Guardamos y Serializamos el Modelo (con esto ya podemos vender nuestro modelo de predicción de texto según lo aprendido por nuestra RNN).

In [73]:
from keras.models import model_from_json
import os
dir_export= '/content/gdrive/MyDrive/Colab Notebooks/RedesNeuronalesRecurrentes/Modelos'
#dir_export= os.path.join(dir_drive)
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open(os.path.join(dir_export,'2RNN_BibliaReinaValera1909_json.json'), 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(dir_export,'2RNN_BibliaReinaValera1909_pesos.hdf5'))
model.save(os.path.join(dir_export,'2RNN_BibliaReinaValera1909_model.h5'))
print("modelo salvado en Drive de google")

modelo salvado en Drive de google


## 17. Cargar modelo serializado

### 17.1 Descargar modelo usando wget

In [77]:
!wget https://github.com/JuanDiegoCastellanos/deep_learning/blob/main/neural_recurrent_network/models/2RNN_BibliaReinaValera1909_model.h5 \
      -O 2RNN_BibliaReinaValera1909_model.h5

--2021-11-23 23:22:13--  https://github.com/JuanDiegoCastellanos/deep_learning/blob/main/neural_recurrent_network/models/2RNN_BibliaReinaValera1909_model.h5
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘2RNN_BibliaReinaValera1909_model.h5’

2RNN_BibliaReinaVal     [ <=>                ] 163.21K  --.-KB/s    in 0.08s   

2021-11-23 23:22:13 (2.12 MB/s) - ‘2RNN_BibliaReinaValera1909_model.h5’ saved [167129]



### 17.2 Descargamo el modelo usando PYRIND & URLLIB (OPCIONAL)

In [78]:
!pip install pyprind

In [79]:
def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size
    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d segundos transcurrido" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()

import urllib.request
url_github_Model='https://github.com/JuanDiegoCastellanos/deep_learning/blob/main/neural_recurrent_network/models/2RNN_BibliaReinaValera1909_model.h5?raw=true'
urllib.request.urlretrieve(url_github_Model,
                           '2RNN_BibliaReinaValera1909_model.h5', 
                           reporthook)

100% | 60 MB | 3.54 MB/s | 17 segundos transcurrido

('2RNN_BibliaReinaValera1909_model.h5',
 <http.client.HTTPMessage at 0x7fa4dc827a90>)

## 18. Instanciación del modelo descargado

In [80]:
new_model = tf.keras.models.load_model(filepath='/content/2RNN_BibliaReinaValera1909_model.h5',custom_objects=None)

In [81]:
df2 = pd.read_csv("https://raw.githubusercontent.com/JuanDiegoCastellanos/deep_learning/main/Datasets/data_vocab.csv")
df2.head()

,num,vocab
0,0,
1,1,0
2,2,1
3,3,2
4,4,3


In [61]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores/cantidad de texto generaremos
  num_generate=500
  #convertimos el texto en números
  input_eval  = [char2idx[s] for s in start_string]
  input_eval  = tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.2  #(0.0 a  1) entre más alta la temperatura más creatividad al modelo, pero tambien más errores ortograficos.
  model.reset_states() #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))

In [88]:
print(generate_text(new_model, start_string=u"inglaterra"))

inglaterra y en medio de ella sobre su cabeza y como la mar y en la soberbia de los palacios de carro de fuego ardiendo 22 y en el campo de los filisteos habian hecho en la casa de jehova y la casa de jehova y la casa de jacob se ha de purificar la casa de jehova y la casa de jehova se encendio contra el en el tiempo de la vida de su rostro y el respondio he aqui que yo envio sobre vosotros y hare cesar de ti tus piedades y hare segun tus mandamientos y sus manos se han mostrado el rey y sus profetas no s
